# PQ Mode - Controllable Source and Load

|  | Mode | Description |
| --- | --- | --- |
| 1 | "Swing" | Ideal voltage source without dynamics (i.e., an infinite bus) |
| 2 | __"PQ"__ | __Grid following controllable source/load (real and imaginary power)__ |
| 3 | "Droop" | Simple grid forming with power balancing through a droop mechanism |
| 4 | "Synchronverter" or "VSG"|  Grid forming control mimicking a generator, i.e., virtual synchronous generator |

- The control structure of power electronic inverters can be divided into cascading levels. 
- The lowest of these employed in the package is an "inner" current control loop, on top of which all other levels are built.

## Summary

- The following example is intended to introduce you to the control mode which will enable the inverter to act like a controllable source or load.
- The mode takes as input the active power (P, W) and the reactive power (Q, VA) as set points.
- Most solar photovoltaic resources, wind power plants and variable loads can be represented by this mode.
- An inverter in this control mode must be placed in a network with other "grid-forming" sources (e.g., swing, droop, or VSG) as it is "grid following".

### Theory

An inverter in the PQ mode is effectively controlled as a current supply, only ever regulating the current exchanged with the grid. This current control loop operates in the direct-quadrature-zero (DQ0) frame, and employs over voltage and current limitations to simulate the protection of the switches (not depicted). This limitations cause non-linear saturation effects when the inverter is pushed to extreme. 


![](figures/current_control.jpg)

The main disadvantage of an inverter in this mode is that the output voltage is maintained by the external network, which means that the inverter needs to synchronize to the network by making use of a phase-locked-loop (PLL). The PLL extracts the angle (and frequency) of the positive phase sequence voltage measured after the first filter inductor. 

A possible disadvantage to the PQ mode is that the inverter may continue injecting currents into the grid when there is a network fault, leading to excessively high voltages. By the same token, when the network voltage is low, the inverter will continue drawing the same amount of power, further deteriorating the voltage profile. Other modes, such as the "Droop" and in particular the "VSG" mode can alleviate some of these issues, providing control structures which regulate both the grid frequency and voltage.  

![](figures/PQ_mode.jpg)


In [1]:
using ElectricGrid;

_______________________________________________________________________________
### Network and Control Configuration 

The following example network structure is considered:
- The PQ-controlled inverter is connected to an external network represented by a source in "Swing" mode.
- The "strength" of this external network is quantifiable by its _fault_ _level_.
- The effective impedance of the external network can be computed from the short-circuit power, the X/R ratio, and the nominal rms voltage.
- Setting I_kp and I_ki (the PI gains) is optional.
- If not provided they will be automatically determined via a loop-shaping method.
- The kp's and ki's for all PI controllers are equal.

With this configuration, a time domain simulation is conducted below.

In [2]:
# total run time, seconds
t_end = 0.16     

# Connectivity Matrix
CM = [ 0. 1.
        -1. 0.]     

R, L = FaultLevel(300e3, 0.5, 230) # short-circuit power [VA], X/R ratio, nominal rms voltage [V]

parameters = Dict{Any, Any}(
        "source" => Any[
                        Dict{Any, Any}("pwr"    => 300e3,    
                                        "mode"  => "Swing", 
                                        "R1"    => R,       # Equivalent external network resistance [Ω]
                                        "L1"    => L)       # Equivalent external network inductance [H] 
                        Dict{Any, Any}("pwr"    => 100e3,   # Rated apparent power [VA]
                                        "mode"  => "PQ",    # Controller mode
                                        "p_set" => 50e3,    # Real power set point (generating) [W] 
                                        "q_set" => -40e3,   # Imaginary power set point (inductive) [VAi]  
                                        "I_kp"  => 0.01,    # Current proportional gain [V/A] (**optional**)
                                        "I_ki"  => 5.0)     # Current integral gain [V/A⋅s] (**optional**)
                        ],
        "cable"   => Any[
                        Dict{Any, Any}("R"  => 0.1, 
                                        "L" => 0.25e-3, 
                                        "C" => 0.1e-4),
                        ],
        "grid"   => Dict{Any, Any}("process_start" => 0.06) # Time to wait before accepting p_set and q_set [s]
    );

env = ElectricGridEnv(CM = CM, parameters = parameters, t_end = t_end, verbosity = 2);

agents = SetupAgents(env);

hook = Simulate(agents, env);

┌ Info: Normalization is done based on the defined parameter limits.
└ @ ElectricGrid c:\Users\SeptimusBoshoff\OneDrive - Power System Dynamics\Documents\Doctor of Philosophy\Paderborn Projects\Repos\dare\src\electric_grid_env.jl:331
┌ Info: Time simulation run time: 0.16 [s] ~> 1601 steps
└ @ ElectricGrid c:\Users\SeptimusBoshoff\OneDrive - Power System Dynamics\Documents\Doctor of Philosophy\Paderborn Projects\Repos\dare\src\electric_grid_env.jl:332


_______________________________________________________________________________
### Low-Level Rendering

In [ ]:
RenderHookResults(hook = hook, 
                    states_to_plot  = ["source2_i_L1_a"], # Inductor current [A]
                    actions_to_plot = ["source2_u_a"],    # Inverter voltage [V]
                    )

_______________________________________________________________________________
### High-Level Rendering

In [ ]:
RenderHookResults(hook = hook, 
                    states_to_plot  = [], 
                    actions_to_plot = [],  
                    v_mag_inv       = [2], # Scaled L₂ norm in αβγ coordinates [V]
                    power_p_inv     = [2], # Real power [W]
                    power_q_inv     = [2], # Imaginary power [VA]
                    angles          = [2], # Relative angle [degrees]
                    freq            = [2], # Angular velocity [Hz]
                    )

_______________________________________________________________________________
### Analysis

- The plot shows the instantaneous 3-phase real and imaginary power delivered by the inverter. 
- At 0.06 s the real and imaginary power set points take effect.
- After the transients have died away the inverter settles down to the specified power set points.
- The inverter's phase-locked loop frequency and angle are indicators of the control stability.
- The stability of the inverter decreases with the short-circuit power (i.e., fault level).

_______________________________________________________________________________
### References

- D. Weber, S. Heid, H. Bode, J. H. Lange, E. Hüllermeier and O. Wallscheid, "Safe Bayesian Optimization for Data-Driven Power Electronics Control Design in Microgrids: From Simulations to Real-World Experiments," in IEEE Access, vol. 9, pp. 35654-35669, 2021, doi: 10.1109/ACCESS.2021.3062144.